In [1]:
import pandas as pd
import numpy as np

In [2]:
sec = pd.read_csv('../sec_sentiment.csv')
catalyst = pd.read_csv('./out/catalyst_sentiment_final.csv')

In [3]:
catalyst['date'] = pd.to_datetime(catalyst['date'], format='%Y-%m-%d')
sec['decyear'] = sec.year + (sec.quarter.apply(lambda x: int(x[-1:])) / 4)
catalyst['decyear'] = catalyst.set_index('date').index.year
catalyst['decyear'] = catalyst.decyear + (catalyst.set_index('date').index.quarter / 4)
catalyst['year'] = catalyst.set_index('date').index.year
catalyst['quarter'] = catalyst.set_index('date').index.quarter
catalyst['quarter'] = catalyst.quarter.apply(lambda x: f'QTR{x}')

In [4]:
sec.head()

,ticker,year,quarter,filing,sentiment_score,decyear
0,HGEN,2021,QTR2,.07. Submission of Matters to a Vote of Securi...,0.027353,2021.50
1,ABEO,2013,QTR4,.02. Departure of Directors or Certain Officer...,0.015559,2014.00
2,EGRX,2017,QTR3,.01 Entry into a Material Definitive Agreement...,0.146894,2017.75
3,AXGN,2021,QTR4,.02 Results of Operations and Financial Condit...,0.109859,2022.00
4,ENTA,2022,QTR2,0001193125-22-178106.txt : 20220622 8-K 1 d356...,0.003334,2022.50


In [5]:
catalyst.head()

,ticker,date,shifted,decyear,year,quarter
0,JAZZ,2010-10-11,-0.159998,2011.00,2010,QTR4
1,ASRT,2011-01-28,0.901584,2011.25,2011,QTR1
2,ASRT,2011-04-29,0.598187,2011.50,2011,QTR2
3,SGEN,2011-08-19,0.313102,2011.75,2011,QTR3
4,PCRX,2011-10-31,0.633965,2012.00,2011,QTR4


In [6]:
agg_scores = list()
for i in range(len(catalyst)):
    sample = catalyst.iloc[i]
    score = sec.loc[(sec.ticker == sample.ticker) &
                    ((sec.decyear <= sample.decyear) &
                     (sec.decyear >= (sample.decyear - 1)))]['sentiment_score'].mean()
    agg_scores.append(score)

In [7]:
catalyst['sec_score'] = agg_scores
catalyst = catalyst.rename({'shifted': 'bio_score'}, axis=1)

In [8]:
catalyst = catalyst.drop(['decyear', 'year', 'quarter'], axis=1)
catalyst.dropna(axis=0, inplace=True)

In [10]:
catalyst.head()

,ticker,date,bio_score,sec_score
0,JAZZ,2010-10-11,-0.159998,0.021264
3,SGEN,2011-08-19,0.313102,0.035399
4,PCRX,2011-10-31,0.633965,0.097330
5,INCY,2011-11-16,0.878714,0.067742
6,HALO,2011-12-08,0.044723,0.126709


In [11]:
catalyst.to_csv('./out/sentiment_scores.csv', index=False)